In [1]:
from llama_index.experimental.query_engine import PandasQueryEngine 
from llama_index.core.query_pipeline import QueryPipeline as QP, Link, InputComponent
from llama_index.experimental.query_engine import PandasInstructionParser
from llama_index.llms.ollama import Ollama
import pandas as pd
from llama_index.core.prompts import PromptTemplate

In [2]:
#url = "https://raw.githubusercontent.com/BUFONJOKER/icc-cwc-dataset/main/icc_cwc.csv"
url = r'C:\icc_cwc.csv'
df = pd.read_csv(url)
#df.to_csv("icc_cwc.csv", index=False)
df["Start Date"] = df["Start Date"].astype(str)
df["Match"] = df["Team 1"] + " vs " + df["Opposition"]
df["team1_runs"] = df["Score"].str.extract(r"(\d+)").astype(int)
df["team2_runs"] = df["Opposition Score"].str.extract(r"(\d+)").astype(int)
df.head(500)

,Team 1,Score,Overs,RPO,Inns,Result,Opposition,Opposition Score,Opposition Overs,Opposition RPO,Opposition Innings,Ground,Start Date,Match,team1_runs,team2_runs
0,England,334/4,60.0,5.56,1,won,v India,132/3,60.0,2.20,2,Lord's,7-Jun-75,England vs v India,334,132
1,New Zealand,309/5,60.0,5.15,1,won,v East Africa,128/8,60.0,2.13,2,Birmingham,7-Jun-75,New Zealand vs v East Africa,309,128
2,Australia,278/7,60.0,4.63,1,won,v Pakistan,205,53.0,3.86,2,Leeds,7-Jun-75,Australia vs v Pakistan,278,205
3,Sri Lanka,86,37.2,2.30,1,lost,v West Indies,87/1,20.4,4.20,2,Manchester,7-Jun-75,Sri Lanka vs v West Indies,86,87
4,England,266/6,60.0,4.43,1,won,v New Zealand,186,60.0,3.10,2,Nottingham,11-Jun-75,England vs v New Zealand,266,186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,England,337/9,50.0,6.74,1,won,v Pakistan,244,43.3,5.60,2,Eden Gardens,11-Nov-23,England vs v Pakistan,337,244
489,India,410/4,50.0,8.20,1,won,v Netherlands,250,47.5,5.22,2,Bengaluru,12-Nov-23,India vs v Netherlands,410,250
490,India,397/4,50.0,7.94,1,won,v New Zealand,327,48.5,6.69,2,Wankhede,15-Nov-23,India vs v New Zealand,397,327
491,South Africa,212,49.4,4.26,1,lost,v Australia,215/7,47.2,4.54,2,Eden Gardens,16-Nov-23,South Africa vs v Australia,212,215


In [3]:
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. PRINT ONLY THE EXPRESSION.\n"
    "5. Do not quote the expression.\n"
)
pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is df.\n"
    "This is the result of print(df.head()):\n"
    "{df_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

In [4]:
pandas_prompt=PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str,df_str=df.tail(5)
)
llm=Ollama(model="llama3:8b", request_timeout=120)
query_engine = PandasQueryEngine(df=df, llm=llm)
response_synthesis_prompt=PromptTemplate(response_synthesis_prompt_str)
pandas_output_parser = PandasInstructionParser(df)

In [5]:
qp = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=True,
)

qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])

qp.add_links([
    Link("input", "response_synthesis_prompt", dest_key="query_str"),
    Link("llm1", "response_synthesis_prompt", dest_key="pandas_instructions"),
    Link("pandas_output_parser", "response_synthesis_prompt", dest_key="pandas_output"),
])

qp.add_link("response_synthesis_prompt", "llm2")

C:\Users\premp\AppData\Local\Temp\ipykernel_9400\1400129676.py:1: DeprecationWarning: Call to deprecated class QueryPipeline. (QueryPipeline has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

It is recommended to switch to the Workflows API for a more flexible and powerful experience.

See the docs for more information workflows: https://docs.llamaindex.ai/en/stable/understanding/workflows/)
  qp = QP(


INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


In [6]:
response=qp.run(
    query_str="what is the match held on 15-Nov-23?",
)

> Running module input with input: 
query_str: what is the match held on 15-Nov-23?

> Running module pandas_prompt with input: 
query_str: what is the match held on 15-Nov-23?

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is df.
This is the result of print(df.head()):
           Team 1  Score  Overs   RPO  Inns Result     Opposition  \
488    ...

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
> Running module pandas_output_parser with input: 
input: assistant: df.loc[(df['Start Date'] == '15-Nov-23') & (df['Match'].str.contains('vs'))]['Match']

> Running module response_synthesis_prompt with input: 
query_str: what is the match held on 15-Nov-23?
pandas_instructions: assistant: df.loc[(df['Start Date'] == '15-Nov-23') & (df['Match'].str.contains('vs'))]['Match']
pandas_output: 490    India vs v New Zealand


In [7]:
print(response.message.content)

The match held on November 15, 2023 is between India and New Zealand.
